# Order Covers

In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root # type: ignore
ROOT = from_root()

import pandas as pd # type: ignore
from loguru import logger # type: ignore

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util # type: ignore

util.startLoop()

logger.add(sink=ROOT / "log" / "snp.log", mode="w")

# Prepare the orders

In [ ]:
from covers import process_covered_positions

_, _, dfcov = process_covered_positions()

In [13]:
## Get open orders
from src.snp import get_ib, get_open_orders, quick_pf

# check open orders
with get_ib('SNP') as ib:
    dfo = get_open_orders(ib)
    dfp = quick_pf(ib)

In [14]:
longs = dfp[(dfp.secType == 'STK') & (dfp.position > 0)].symbol.to_list()
shorts = dfp[(dfp.secType == 'STK') & (dfp.position < 0)].symbol.to_list()


In [15]:
existing_long_order_symbols = dfo[dfo.symbol.isin(longs) & (dfo.right == 'C')].symbol.to_list()
existing_short_order_symbols = dfo[dfo.symbol.isin(shorts) & (dfo.right == 'P')].symbol.to_list()

In [ ]:
symbols_to_remove = existing_long_order_symbols + existing_short_order_symbols
dfcov[~dfcov.ib_symbol.isin(symbols_to_remove)]

In [20]:
from ib_async import LimitOrder
from snp import get_prec

def make_ib_orders(df: pd.DataFrame) -> tuple:
    """Make (contract, order) tuples"""

    contracts = df.contract.to_list()
    orders = [
        LimitOrder(action="SELL", totalQuantity=abs(int(q)), lmtPrice=get_prec(p, 0.01))
        for q, p in zip(df.lot, df.xPrice)
    ]

    cos = tuple((c, o) for c, o in zip(contracts, orders))

    return cos

In [ ]:
# make the lots
dfcov['lot'] = abs(dfcov.position/100).astype('int')
dfcov.head()

In [ ]:
# Make contract orders of the target
cos = make_ib_orders(dfcov)
cos

# PLACE THE ORDERS
